# Low dim

In [1]:
import os
import json
from robomimic.config import config_factory

filter_key = "worse_better"

milestones = [500]
mask_names = [['worse'], ['better']]

project_name = 'can_worse_better_lowdim'

sequence_lens = {'bc': 1, 'bc_rnn': 10, 'bcq': 1}
config_names = {'bc': 'bc', 'bc_rnn': 'bc', 'bcq': 'bcq'}

config_save_root = '/vision/shuhan/projects/imitation/mix_demo/curr_configs'
result_save_root = '/vision/shuhan/projects/imitation/mix_demo/results'

seeds = [2, 3]

for seed in seeds:
  for model_name, config_name in config_names.items():
    config = config_factory(config_name)

    config.project = project_name

    config.train.data = '/vision/shuhan/projects/imitation/mix_demo/datasets/can/mh/low_dim.hdf5'
    config.train.seed = seed
    config.train.hdf5_cache_mode = 'low_dim'
    config.train.hdf5_filter_key = filter_key
    config.train.seq_length = sequence_lens[model_name]
    config.experiment.rollout.rate = 250

    if model_name == 'bc_rnn':
      config.algo.rnn.enabled = True
    
    for curr_mode in ['uniform', 'demo_quality_500']:
      
      if curr_mode == 'demo_quality_500':
        config.curriculm.name = "demo_quality"
      else:
        config.curriculm.name = curr_mode

      if curr_mode == 'demo_quality_500':
        config.curriculm.demo_quality.milestones = milestones
        config.curriculm.demo_quality.mask_names = mask_names
        config.curriculm.demo_quality.sample_weights = [1.0, 0.0]
      
      config.train.output_dir = os.path.join(result_save_root, project_name, model_name)
      config.experiment.name = '{}_{}'.format(model_name, curr_mode)

      config_save_folder = os.path.join(config_save_root, project_name, model_name)
      os.makedirs(config_save_folder, exist_ok=True)
      config_save_file = os.path.join(config_save_folder, '{}_seed_{}.json'.format(curr_mode, seed))

      with open(config_save_file, 'w') as f:
        json.dump(config, f, indent=4, separators=(',', ': '))

# Image

In [ ]:
import os
import json
from robomimic.config import config_factory

filter_key = "worse_better"

milestones = [150]
mask_names = [['worse'], ['better']]

project_name = 'can_worse_better_image'

sequence_lens = {'bc': 1, 'bc_rnn': 10, 'bcq': 1}
config_names = {'bc': 'bc', 'bc_rnn': 'bc', 'bcq': 'bcq'}

config_save_root = '/vision/shuhan/projects/imitation/mix_demo/curr_configs'
result_save_root = '/vision/shuhan/projects/imitation/mix_demo/results'

for model_name, config_name in config_names.items():
  config = config_factory(config_name)

  config.project = project_name

  config.train.num_data_workers = 0
  config.train.batch_size = 16
  config.train.num_epochs = 600
  config.train.data = '/vision/shuhan/projects/imitation/mix_demo/datasets/can/mh/image.hdf5'
  config.train.hdf5_cache_mode = 'all'
  config.train.hdf5_filter_key = filter_key
  config.experiment.rollout.rate = 40
  config.experiment.keep_all_videos = True
  config.experiment.epoch_every_n_steps = 500
  config.train.seq_length = sequence_lens[model_name]

  config.observation.modalities.obs.low_dim = ["robot0_eef_pos", "robot0_eef_quat", "robot0_gripper_qpos"]
  config.observation.modalities.obs.rgb = ["agentview_image", "robot0_eye_in_hand_image"]

  if model_name == 'bc_rnn':
    config.algo.rnn.enabled = True
  
  for curr_mode in ['uniform', 'demo_quality_500']:
    
    if curr_mode == 'demo_quality_500':
      config.curriculm.name = "demo_quality"
    else:
      config.curriculm.name = curr_mode

    if curr_mode == 'demo_quality_500':
      config.curriculm.demo_quality.milestones = milestones
      config.curriculm.demo_quality.mask_names = mask_names
      config.curriculm.demo_quality.sample_weights = [1.0, 0.0]
    
    config.train.output_dir = os.path.join(result_save_root, project_name, model_name)
    config.experiment.name = '{}_{}'.format(model_name, curr_mode)

    config_save_folder = os.path.join(config_save_root, project_name, model_name)
    os.makedirs(config_save_folder, exist_ok=True)
    config_save_file = os.path.join(config_save_folder, '{}.json'.format(curr_mode))

    with open(config_save_file, 'w') as f:
      json.dump(config, f, indent=4, separators=(',', ': '))

# Make seed file

In [12]:
import os
import json

folder = '/vision/shuhan/projects/imitation/mix_demo/curr_configs/can_worse_better_lowdim/'
models = ['bc', 'bc_rnn']
file_names = ['later_var', 'no_pretrain', 'square_pretrain_500']
seeds = [2, 3]

for model in models:
  for file_name in file_names:
    file_path = os.path.join(folder, model, file_name+'.json')

    with open(file_path, 'r') as f:
      config = json.load(f)

    for seed in seeds:
      config['train']['seed'] = seed

      save_file = os.path.join(folder, model, file_name+'_seed_'+str(seed)+'.json')

      with open(save_file, 'w') as f:
        json.dump(config, f, indent=4, separators=(',', ': '))

In [1]:
import os
import json

folder = '/vision/shuhan/projects/imitation/mix_demo/curr_configs/can_worse_better_image/'
models = ['bc', 'bc_rnn', 'bcq']
file_names = ['uniform', 'demo_quality_500']
seeds = [2, 3]

for model in models:
  for file_name in file_names:
    file_path = os.path.join(folder, model, file_name+'.json')

    with open(file_path, 'r') as f:
      config = json.load(f)

    for seed in seeds:
      config['train']['seed'] = seed

      save_file = os.path.join(folder, model, file_name+'_seed_'+str(seed)+'.json')

      with open(save_file, 'w') as f:
        json.dump(config, f, indent=4, separators=(',', ': '))

# Enable GMM for BC and BC_rnn

In [1]:
import os
import glob
import json

source_folder = '/vision/shuhan/projects/imitation/mix_demo/curr_configs/can_worse_better_lowdim'
save_folder = '/vision/shuhan/projects/imitation/mix_demo/curr_configs/can_worse_better_lowdim_gmm'

output_root = '/vision/shuhan/projects/imitation/mix_demo/results'

model_names = ['bc', 'bc_rnn']

for model in model_names:
  config_files = glob.glob(os.path.join(source_folder, model, '*.json'))
  config_files = [file for file in config_files if 'seed' not in file]

  for file in config_files:
    with open(file, 'r') as f:
      config = json.load(f)
    
    config['algo']['gmm']['enabled'] = True
    config['project'] = 'can_worse_better_lowdim_gmm'

    config['train']['output_dir'] = os.path.join(output_root, config['project'])


    file_name = file.split('/')[-1]
    
    model_save_folder = os.path.join(save_folder, model)
    os.makedirs(model_save_folder, exist_ok=True)

    save_file = os.path.join(model_save_folder, file_name)
    with open(save_file, 'w') as f:
      json.dump(config, f, indent=4, separators=(',', ': '))

# Create configs for Square task

In [2]:
import os
import glob
import json

source_folder = '/vision/shuhan/projects/imitation/mix_demo/curr_configs/can_worse_better_lowdim'
save_folder = '/vision/shuhan/projects/imitation/mix_demo/curr_configs/square_worse_better_lowdim_gmm'

output_root = '/vision/shuhan/projects/imitation/mix_demo/results'

model_names = ['bc', 'bc_rnn', 'bcq']

for model in model_names:
  config_files = glob.glob(os.path.join(source_folder, model, '*.json'))
  config_files = [file for file in config_files if 'seed' not in file]

  for file in config_files:
    if 'separate' in file:
      continue

    with open(file, 'r') as f:
      config = json.load(f)
    
    if 'bcq' not in model:
      config['algo']['gmm']['enabled'] = True
    
    config['project'] = 'square_worse_better_lowdim_gmm'
    config['experiment']['rollout']['horizon'] = 500
    config['experiment']['rollout']['rate'] = 200
    config['train']['data'] = '/vision/shuhan/projects/imitation/mix_demo/datasets/square/mh/low_dim.hdf5'

    config['train']['output_dir'] = os.path.join(output_root, config['project'])

    if 'square_pretrain' in file:
      config['pretrain']['data'] = '/vision/shuhan/projects/imitation/mix_demo/datasets/can/mh/low_dim.hdf5'
      file_name = 'can_pretrain_500.json'
      config['experiment']['name'] = model + '_can_pretrain_500'
    else:
      file_name = file.split('/')[-1]
    
    model_save_folder = os.path.join(save_folder, model)
    os.makedirs(model_save_folder, exist_ok=True)

    save_file = os.path.join(model_save_folder, file_name)
    with open(save_file, 'w') as f:
      json.dump(config, f, indent=4, separators=(',', ': '))

# Create action variance analysis configs

In [1]:
import os
import glob
import json

config_root = '/vision/shuhan/projects/imitation/mix_demo/curr_configs'
save_folder = '/vision/shuhan/projects/imitation/mix_demo/curr_configs/action_variance_analysis'

model_names = ['bc', 'bc_rnn']
task_names = ['can', 'square']
exp_names = ['later_var', 'uniform', 'no_pretrain', 'demo_quality_500']

output_root = '/vision/shuhan/projects/imitation/mix_demo/results'


for task in task_names:
  task_folder = os.path.join(save_folder, task)

  for model in model_names:
    model_folder = os.path.join(task_folder, model)
    os.makedirs(model_folder, exist_ok=True)

    source_folder = os.path.join(config_root, '{}_worse_better_lowdim_gmm'.format(task), model)

    for exp_name in exp_names:
      config_file = os.path.join(source_folder, exp_name+'.json')

      with open(config_file, 'r') as f:
        config = json.load(f)
      
      config['project'] = 'action_variance_{}'.format(task)
      config['train']['output_dir'] = os.path.join(output_root, config['project'])
      config['experiment']['action_variance'] = True
      config['experiment']['action_variance_keys'] = ["worse", "better"]
      config['experiment']['validation_epoch_every_n_steps'] = 50

      save_file = os.path.join(model_folder, exp_name+'.json')
      with open(save_file, 'w') as f:
        json.dump(config, f, indent=4, separators=(',', ': '))